In [1]:
import os 
import json
import re

In [2]:
def parse_execution_history(history_string):
    history_string = history_string.replace("Historical Execution: ", "")
    historty_splits = re.split(r'(\(fail\)|\(success\))\s', history_string)

    parsed_steps = []

    for i in range(0, len(historty_splits)-1, 2):
        result = 'success' if 'success' in historty_splits[i+1] else 'fail'
        action_parts = re.sub(r'\(.*?\)', '', historty_splits[i], 1).strip().lstrip('[').rstrip(']').split(',')
        
        parsed_step = [
            action_parts[0].strip(),
            action_parts[1].strip().lower() if len(action_parts) > 1 else "None",
            action_parts[2].strip().lower() if len(action_parts) > 2 else "None",
            result
        ]
        parsed_steps.append(parsed_step)

    return parsed_steps

In [ ]:
exp_name = "Qwen2VL"
exp_path = os.path.join("exp", exp_name)

eval_list = []

for index in os.listdir(exp_path):
    for i in range(1, 4):
        conv_path = os.path.join(exp_path, index, f"{i}/conversation.json")
        if not os.path.exists(conv_path):
            continue
        with open(conv_path, 'r') as file:
            conv = json.load(file)

        exe_dict = {}
        exe_dict["index"] = index
        exe_dict["times"] = i 

        add_end = 0
        if "End" in conv[-1]:
            add_end = 1
        history = conv[-1].split('\n')[3]
        exe_dict["history"] = parse_execution_history(history)
        if add_end == 1:
            exe_dict['history'].append(["End", "None", "None", "success"])
        exe_dict['step_len'] = len(exe_dict['history'])
        eval_list.append(exe_dict)

print(len(eval_list))

300


In [ ]:
def load_name_mapping():
        with open('hab-mobile-manipulation/name_dict.txt', 'r') as file:
            content = file.read()
        lines = content.split('\n')
        print(lines[-1])
        
        name_dict = {}
        for i in range(0, len(lines), 3):
            value = lines[i].strip().strip(':')
            key = lines[i + 1].strip()
            keys = key.split('/')
            for i in keys:
                name_dict[i] = value

        return name_dict
    # mapping item name
mapping_dict = load_name_mapping()

def match_action(action, keypoint, mapping_dict):
        if action[3] != 'success':
               return False

        key_action = [element.strip() for element in keypoint.strip('[').strip(']').split(',')]
        if len(key_action) == 2:
            key_action.append("None")
        if action[0] == key_action[0]:
                if key_action[0] == "End":
                    return True
                obj1 = mapping_dict[action[1]]
                obj2 = mapping_dict[key_action[1]]
                if obj1 == obj2:
                       obj11 = "None" if action[2] == "None" else mapping_dict[action[2]]
                       obj22 = "None" if key_action[2] == "None" else mapping_dict[key_action[2]]
                       if obj11 == obj22:
                              return True
        return False

def match(list1, list2):
    if len(list2) == 0:
        return 0
    index1 = 0
    index2 = 0
    
    while index1 < len(list1) and index2 < len(list2):
        
        if match_action(list1[index1], list2[index2], mapping_dict):
            index2 += 1
        index1 += 1
    
    
    return index2 / len(list2)


In [ ]:
for eval_node in eval_list:
    task_id = eval_node['index']
    history = eval_node['history']

    if 1 <= int(task_id) <= 90:
        data_dir = "EMMOE-100/data/train"
    else:
        data_dir = "EMMOE-100/data/test"

    with open(os.path.join(data_dir, str(task_id), "keypath.json"), 'r') as file:
        keypaths = json.load(file)

    tp = 0
    for keypath in keypaths:
        tp_tmp = match(history, keypath)
        tp = max(tp_tmp, tp)
            
    eval_node['tp'] = tp

In [6]:
train_cnt = 0
test_cnt = 0
train_success = 0
test_success = 0
train_tp = 0
test_tp = 0

In [7]:
for node in eval_list:
    task_id = node['index']
    if 1 <= int(task_id) <= 90:
        train_cnt += 1
        train_tp += node['tp']
        if node['tp'] == 1:
            train_success += 1
    else:
        test_cnt += 1
        test_tp += node['tp']
        if node['tp'] == 1:
            test_success += 1        

In [8]:
train_avg_tp = train_tp / train_cnt
test_avg_tp = test_tp / test_cnt
train_sr = train_success / train_cnt
test_sr = test_success / test_cnt

In [9]:
print(train_sr, train_avg_tp, test_sr, test_avg_tp)

0.011111111111111112 0.15784391534391543 0.0 0.23452380952380958


In [10]:
all_sr = (train_success + test_success)/(train_cnt + test_cnt)
all_tp = (train_tp + test_tp)/(train_cnt + test_cnt)
print(all_sr,all_tp)

0.01 0.16551190476190483
